# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 21 2022 11:15AM,242432,19,8304146,"Graystone, LLC",Released
1,Jul 21 2022 11:14AM,242422,15,PNC377493A,"Person & Covey, Inc.",Released
2,Jul 21 2022 11:14AM,242422,15,PNC377494A,"Person & Covey, Inc.",Released
3,Jul 21 2022 11:14AM,242422,15,PNC377496A,"Person & Covey, Inc.",Released
4,Jul 21 2022 11:14AM,242422,15,PNC377497A,"Person & Covey, Inc.",Released
5,Jul 21 2022 11:14AM,242422,15,PNC377498A,"Person & Covey, Inc.",Released
6,Jul 21 2022 11:14AM,242422,15,PNC377499A,"Person & Covey, Inc.",Released
7,Jul 21 2022 11:14AM,242422,15,PNC377500A,"Person & Covey, Inc.",Released
8,Jul 21 2022 11:14AM,242422,15,PNC377501A,"Person & Covey, Inc.",Released
9,Jul 21 2022 11:14AM,242422,15,PNC377503A,"Person & Covey, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,242426,Released,1
38,242427,Released,1
39,242429,Released,12
40,242431,Released,1
41,242432,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
242426,NaN,1.0
242427,NaN,1.0
242429,NaN,12.0
242431,NaN,1.0
242432,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
242325,0.0,1.0
242328,0.0,1.0
242336,0.0,1.0
242340,0.0,1.0
242348,0.0,78.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
242325,0,1
242328,0,1
242336,0,1
242340,0,1
242348,0,78


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,242325,0,1
1,242328,0,1
2,242336,0,1
3,242340,0,1
4,242348,0,78


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,242325,,1
1,242328,,1
2,242336,,1
3,242340,,1
4,242348,,78


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 21 2022 11:15AM,242432,19,"Graystone, LLC"
1,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc."
16,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc."
17,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc."
29,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC"
32,Jul 21 2022 10:56AM,242424,15,"Mizner Bioscience, LLC"
34,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc."
35,Jul 21 2022 10:54AM,242426,19,ACG North America LLC
36,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC"
41,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 21 2022 11:15AM,242432,19,"Graystone, LLC",,1
1,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",,15
2,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",,1
3,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",,12
4,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",,3
5,Jul 21 2022 10:56AM,242424,15,"Mizner Bioscience, LLC",,2
6,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",,1
7,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,,1
8,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",,5
9,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 11:15AM,242432,19,"Graystone, LLC",1,
1,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",15,
2,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",1,
3,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",12,
4,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",3,
5,Jul 21 2022 10:56AM,242424,15,"Mizner Bioscience, LLC",2,
6,Jul 21 2022 10:54AM,242423,15,"Chadwick Pharmaceuticals, Inc.",1,
7,Jul 21 2022 10:54AM,242426,19,ACG North America LLC,1,
8,Jul 21 2022 10:53AM,242421,15,"Carwin Pharmaceutical Associates, LLC",5,
9,Jul 21 2022 10:52AM,242427,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 11:15AM,242432,19,"Graystone, LLC",1,
1,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",15,
2,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",1,
3,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",12,
4,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 11:15AM,242432,19,"Graystone, LLC",1.0,NaN
1,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",15.0,NaN
2,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",1.0,NaN
3,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",12.0,NaN
4,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 21 2022 11:15AM,242432,19,"Graystone, LLC",1.0,0.0
1,Jul 21 2022 11:14AM,242422,15,"Person & Covey, Inc.",15.0,0.0
2,Jul 21 2022 11:04AM,242431,21,"NBTY Global, Inc.",1.0,0.0
3,Jul 21 2022 10:59AM,242429,15,"Alliance Pharma, Inc.",12.0,0.0
4,Jul 21 2022 10:57AM,242425,15,"Virtus Pharmaceuticals, LLC",3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2423914,135.0,21.0
12,242336,1.0,0.0
15,1454544,38.0,0.0
16,1212032,8.0,0.0
19,1454358,17.0,2.0
20,484761,81.0,0.0
21,1939029,8.0,0.0
22,242417,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2423914,135.0,21.0
1,12,242336,1.0,0.0
2,15,1454544,38.0,0.0
3,16,1212032,8.0,0.0
4,19,1454358,17.0,2.0
5,20,484761,81.0,0.0
6,21,1939029,8.0,0.0
7,22,242417,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,135.0,21.0
1,12,1.0,0.0
2,15,38.0,0.0
3,16,8.0,0.0
4,19,17.0,2.0
5,20,81.0,0.0
6,21,8.0,0.0
7,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,135.0
1,12,Released,1.0
2,15,Released,38.0
3,16,Released,8.0
4,19,Released,17.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Executing,21.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
Released,135.0,1.0,38.0,8.0,17.0,81.0,8.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Executing,21.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
1,Released,135.0,1.0,38.0,8.0,17.0,81.0,8.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Executing,21.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
1,Released,135.0,1.0,38.0,8.0,17.0,81.0,8.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()